In [ ]:
from IPython import get_ipython
from IPython.display import display

**Connect google drive**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Requitment**

In [ ]:
!pip install ctransformers
!pip install transformers
!pip install langchain
!pip install langchain-community
!pip install torch
!pip install pypdf
!pip install sentence-transformers
!pip install faiss-cpu
!pip install chainlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/2

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import CTransformers
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from transformers import AutoTokenizer
import time
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# path
vector_db_path = "/content/drive/MyDrive/RAG_model/vectorstores/db_faiss"
embedding_path = "sentence-transformers/all-MiniLM-L6-v2"
model_file = "/content/drive/MyDrive/RAG_model/models/vinallama-7b-chat_q5_0.gguf"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(embedding_path)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

# **Ingestion**

**Define a create data base function**

**From text**

In [ ]:
def create_db_from_text(raw_text, embedding_path, vector_db_path):

  # split the text
  text_spliter = CharacterTextSplitter(
      separator="\n",
      chunk_size = 300,
      chunk_overlap = 50,
      length_function = len
  )

  chunks = text_spliter.split_text(raw_text)
  print(f"From this text we have {len(chunks)} chunks")
  print("Chunks:", chunks[1] + "\n")

  #Tokenize
  print("Number token of chunks:")
  for i, chunk in enumerate(chunks):
    tokens = tokenizer.tokenize(chunk)
    print(f"Chunk {i} has {len(tokens)} tokens")

  # Embedding the chunks
  # Check the embedding_path
  # Load HuggingFace embeddings
  try:
      embedding_model = HuggingFaceEmbeddings(model_name=embedding_path)
  except Exception as e:
      raise RuntimeError(f"Error loading embedding model: {str(e)}")

  # Create the DB
  database = FAISS.from_texts(chunks, embedding_model)
  database.save_local(vector_db_path)
  return database

In [ ]:
# Create database
def create_db_from_pdf(pdf_data_path, embedding_path, vector_db_path):

  # Load PDF
  loader = PyPDFLoader(pdf_data_path)
  documents = loader.load()

  # Spliter
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=300,
                                                 chunk_overlap=50)
  chunks = text_splitter.split_documents(documents)
  print(f"From this pdf we have {len(chunks)} chunks")
  print("Chunks:", chunks[1])
  print("\n")

  #Tokenize
  print("Number token of chunks:")
  for i, chunk in enumerate(chunks):
    tokens = tokenizer.tokenize(chunk.page_content)
    print(f"Chunk {i} has {len(tokens)} tokens")

  # Embedding
  try:
    embedding_model = HuggingFaceEmbeddings(model_name= embedding_path)
  except:
    raise RuntimeError(f"Error loading embedding model: {str(e)}")

  # Create Database
  database = FAISS.from_documents(chunks, embedding_model)
  database.save_local(vector_db_path)
  return database

# **Retrieval**

In [ ]:
# Load the model LLM
def Load_model(model_path):
  # load the local model downloaded
  llm = CTransformers(model = model_file,
                     model_type = 'llama',
                     max_new_tokens = 512,
                     temperature = 0.1
                     )
  return llm

In [ ]:
custom_prompt_template = """<|im_start|>system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác ngắn gọn và chính xác.
{context}
<|im_end|>
<|im_start|>user
{question}<|im_end|>
<|im_start|>assistant
"""

In [ ]:
# Create prompt template
def prompt_template(prompt_template):
    prompt = PromptTemplate(template = prompt_template,
                            input_variables = ['question', 'context'])
    return prompt

# **Generation**

In [ ]:
# QA model functions
def qa_bot():
  # Load db
  embeddings = HuggingFaceEmbeddings(model_name= embedding_path)
  database = FAISS.load_local(vector_db_path,
                              embeddings,
                              allow_dangerous_deserialization=True)

  # Load LLM
  llm = Load_model(model_file)

  # Create prompt template
  prompt = prompt_template(custom_prompt_template)

  # Create QA chain
  qa_info = RetrievalQA.from_chain_type(llm=llm,
                                         chain_type="stuff",
                                         retriever=database.as_retriever(search_kwargs={"k": 2}),
                                         chain_type_kwargs={"prompt": prompt})


  return qa_info

In [ ]:
# output
def final_result(question):
  qa_result = qa_bot()
  response = qa_result.invoke({'query': question})
  return response

# **Deploy**

In [ ]:
# @cl.on_chat_start
# async def start():
#   chain =  qa_bot()
#   # message
#   message = cl.Message(content="Starting the bot...")
#   await message.send()

#   message.content = "Hi, Welcome to the bot. What is your query?"
#   await message.update()

#   cl.user_session.set("chain", chain)

In [ ]:
# @cl.on_message
# async def main(message: cl.Message):
#     chain = cl.user_session.get("chain")
#     cb = cl.AsyncLangchainCallbackHandler(
#         stream_final_answer=True, answer_prefix_tokens=["FINAL", "ANSWER"]
#     )
#     cb.answer_reached = True
#     res = await chain.acall(message.content, callbacks=[cb])
#     answer = res["result"]
#     sources = res["source_documents"]

#     if sources:
#         answer += f"\nSources:" + str(sources)
#     else:
#         answer += "\nNo sources found"

#     await cl.Message(content=answer).send()

# Evalution the RAG

**With raw text**

In [ ]:
raw_text = '''
Mô tả công việc
Nhận yêu cầu thiết kế từ quản lý/ trưởng phòng Kỹ thuật
Đo đạc, khảo sát chi tiết để sản xuất
Triển khai bản vẽ chi tiết (dựa trên kích thước đo đạc thực tế kết hợp với hình ảnh 3D) để sản xuất)
Hỗ trợ các phòng ban liên quan đến kỹ thuật.
Nắm rõ về chất liệu, phụ kiện (Cập nhật và đào tạo thêm).
Ưu tiên nhân sự có thể khảo sát 1 mình, có thể khảo sát ngoài giờ hành chính.
Bóc tách chi tiết sát với hình ảnh 3D.
Có đề xuất thay đổi chi tiết 2D so với 3D nếu thấy hợp lý hơn.
Làm viêc theo sự phân công từ cấp trên
Quyền lợi
Lương Cơ bản là 7000000vnd.
Thưởng lương tháng 13 và thưởng các ngày lễ, trong năm
Được làm việc trong môi trường năng động, có nhiều cơ hội thăng tiến trong nghề nghiệp
Được hưởng đầy đủ phúc lợi xã hội theo quy định của luật lao động như: BHXH, BHYT, BHTN...
Hàng năm xét hiệu xuất công việc và tăng lương định kỳ
Địa điểm làm việc
- Hà Nội: số 48 Trần Kim Xuyến - Yên Hòa, Cầu Giấy
- Hồ Chí Minh: Số 7 Đường Cộng Hòa, phường 4, Tân Bình
'''

In [ ]:
create_db_from_text(raw_text, embedding_path, vector_db_path)

From this text we have 4 chunks
Chunks: Hỗ trợ các phòng ban liên quan đến kỹ thuật.
Nắm rõ về chất liệu, phụ kiện (Cập nhật và đào tạo thêm).
Ưu tiên nhân sự có thể khảo sát 1 mình, có thể khảo sát ngoài giờ hành chính.
Bóc tách chi tiết sát với hình ảnh 3D.
Có đề xuất thay đổi chi tiết 2D so với 3D nếu thấy hợp lý hơn.

Number token of chunks:
Chunk 0 has 107 tokens
Chunk 1 has 112 tokens
Chunk 2 has 82 tokens
Chunk 3 has 117 tokens


<ipython-input-6-3414076e2774>:25: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name=embedding_path)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

**Check response**

In [ ]:
question1 = "Lương cơ bản là bao nhiêu ?"

In [ ]:
response1 = final_result(question1)

In [ ]:
print(response1)

{'query': 'Lương cơ bản là bao nhiêu ?', 'result': 'Lương cơ bản cho vai trò này là 7000000vnd.'}


In [ ]:
question2 = "Địa điểm làm việc ở Hồ Chí Minh là gì ?"

In [ ]:
response2 = final_result(question2)

In [ ]:
print(response2)

{'query': 'Địa điểm làm việc ở Hồ Chí Minh là gì ?', 'result': 'Địa điểm làm việc ở Hồ Chí Minh nằm tại số 7 Đường Cộng Hòa, phường 4, Tân Bình.'}


**With PDF file**

**Create vector database**

In [ ]:
pdf_filepath = '/content/drive/MyDrive/RAG_model/data/Nhân viên kinh doanh.pdf'

In [ ]:
create_db_from_pdf(pdf_filepath, embedding_path, vector_db_path)

From this pdf we have 7 chunks
Chunks: page_content='- Tư vấn và giải đáp các thắc mắc của khách hàng 
- Đề xuất các kế hoạch kinh doanh, phương án hỗ trợ khách hàng nhằm thúc đẩy doanh thu. 
- Duy trì và phát triển mối quan hệ với khách hàng, xây dựng tệp khách hàng tiềm năng.' metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2024-05-08T13:56:13+00:00', 'moddate': '2024-05-08T13:56:13+00:00', 'source': '/content/drive/MyDrive/RAG_model/data/Nhân viên kinh doanh.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}


Number token of chunks:
Chunk 0 has 107 tokens
Chunk 1 has 91 tokens
Chunk 2 has 107 tokens
Chunk 3 has 112 tokens
Chunk 4 has 91 tokens
Chunk 5 has 119 tokens
Chunk 6 has 54 tokens


**Check response**

In [ ]:
question3 = "Lương cơ bản là bao nhiêu ?"

In [ ]:
response3 = final_result(question3)
print(response3)

{'query': 'Lương cơ bản là bao nhiêu ?', 'result': 'Lương cơ bản cho vị trí này thường dao động trong khoảng 6 đến 12 triệu. Tuy nhiên, đây chỉ là khung lương chung và có thể thay đổi tùy thuộc vào các yếu tố khác nhau như kinh nghiệm của ứng viên, nhu cầu thị trường và chính sách của công ty.'}


In [ ]:
question4 = "Địa điểm làm việc ở Hồ Chí Minh là gì ?"

In [ ]:
response4 = final_result(question4)
print(response4)

{'query': 'Địa điểm làm việc ở Hồ Chí Minh là gì ?', 'result': 'Địa điểm làm việc ở Hồ Chí Minh là 239 Tân Sơn Nhì, P. Tân Sơn Nhì, Tân Phú. Thời gian làm việc từ thứ 2 đến thứ 6 với các thời khóa biểu sau đây: Sáng từ 08h00 đến 12h00, Chiều từ 13h00 đến 17h00. Vào thứ 7, thời gian làm việc từ 08h00 đến 12h00. Có hưởng đầy đủ chính sách phúc lợi của công ty bao gồm thưởng hiệu quả công việc, thưởng Tết và du lịch thường niên. Bạn cũng có thể đề xuất ý kiến với cấp quản lý để thực hiện công việc được hiệu quả hơn.'}


# Extension

**Create vector database with Directory**

In [ ]:
def create_db_from_directory(data_path, embedding_path, vector_db_path):
  # Loader
  Loader = DirectoryLoader(data_path,
                           glob= "./*.pdf",
                           loader_cls= PyPDFLoader)
  Documents = Loader.load()
  print(f"Upload {len(Documents)} from directory")

  # Split Documents
  text_spliter = RecursiveCharacterTextSplitter(chunk_size = 300,
                                                chunk_overlap = 50)
  chunks = text_spliter.split_documents(Documents)
  print(f"From this directory we have {len(chunks)} chunks")
  print("Chunks:", chunks[1])
  print("\n")

  #Tokenize
  print("Number token of chunks:")
  for i, chunk in enumerate(chunks):
    tokens = tokenizer.tokenize(chunk.page_content)
    print(f"Chunk {i} has {len(tokens)} tokens")

  # Embedding
  try:
    embedding_model = HuggingFaceEmbeddings(model_name= embedding_path)
  except:
    raise RuntimeError(f"Error loading embedding model: {str(e)}")

  # Create Database
  database = FAISS.from_documents(chunks, embedding_model)
  database.save_local(vector_db_path)
  return database

**Nhận xét:**

Mỗi lần tạo vector data base thì kết quả sẽ bị ghi đè lên file cũ


*   Nhược điểm: Mất thông tin.
*   Cách khắc phục: Mỗi lần gọi hàm  tạo database thì kiểm tra data base đã tồn tại chưa?

1.   Nếu cỏ, mở ra và ghi thêm thông tin vào.
2.   Nếu chưa, tạo một vector data base mới.






In [ ]:
# # Tạo hoặc tải database
#     if os.path.exists(vector_db_path) and os.path.exists(os.path.join(vector_db_path, 'index.faiss')):
#         database = FAISS.load_local(vector_db_path, embedding_model)
#         new_embeddings = embedding_model.embed_documents([text.page_content for text in texts])
#         database.add_texts([text.page_content for text in texts])

#     else:
#         database = FAISS.from_documents(texts, embedding_model)

In [ ]:
data_path = "/content/drive/MyDrive/RAG_model/data"

In [ ]:
create_db_from_directory(data_path, embedding_path, vector_db_path)

Upload 11 from directory
From this directory we have 82 chunks
Chunks: page_content='- Live voice only, not face live 
- Guide, answer, answer customers about features, prices, promotions, put products in the basket or 
order, get discount codes, instructions to view product lists, product uses,  
- Chat with viewers to answer questions, answer viewers' questions' metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2024-05-06T15:06:17+00:00', 'moddate': '2024-05-06T15:06:17+00:00', 'source': '/content/drive/MyDrive/RAG_model/data/Consultant VPBANK Project Finance.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}


Number token of chunks:
Chunk 0 has 54 tokens
Chunk 1 has 55 tokens
Chunk 2 has 75 tokens
Chunk 3 has 43 tokens
Chunk 4 has 57 tokens
Chunk 5 has 52 tokens
Chunk 6 has 61 tokens
Chunk 7 has 54 tokens
Chunk 8 has 55 tokens
Chunk 9 has 75 tokens
Chunk 10 has 43 tokens
Chunk 11 has 57 tokens
Chunk 12 has

**Check response**

In [ ]:
question5 = 'Mô tả công việc của nhân viên kinh doanh ?'

In [ ]:
response5 = final_result(question5)
print(response5)

{'query': 'Mô tả công việc của nhân viên kinh doanh ?', 'result': 'Công việc của một nhân viên kinh doanh thường liên quan đến các nhiệm vụ sau: \n- Tìm hiểu và khảo sát thị trường để xác định nhu cầu, đối tượng khách hàng tiềm năng \n- Giới thiệu sản phẩm hoặc dịch vụ cho khách hàng tiềm năng, giải quyết thắc mắc của họ, và xây dựng mối quan hệ \n- Thuyết phục và tư vấn khách hàng về các chính sách bán hàng và khuyến mãi của công ty. \n- Theo dõi và chăm sóc khách hàng hiện tại để đảm bảo sự hài lòng liên tục \n- Tham gia vào các hoạt động phát triển khách hàng và tiếp thị, như quảng cáo, trưng bày sản phẩm, hội chợ triển lãm... \n- Ghi chép thông tin liên lạc với khách hàng và phản hồi của họ \n- Báo cáo quản lý về tiến độ bán hàng, số liệu thống kê và các nhiệm vụ công việc khác theo yêu cầu. \n<|im_end|>'}


In [ ]:
print(response5['query'])
print(response5['result'])

Mô tả công việc của nhân viên kinh doanh ?
Công việc của một nhân viên kinh doanh thường liên quan đến các nhiệm vụ sau: 
- Tìm hiểu và khảo sát thị trường để xác định nhu cầu, đối tượng khách hàng tiềm năng 
- Giới thiệu sản phẩm hoặc dịch vụ cho khách hàng tiềm năng, giải quyết thắc mắc của họ, và xây dựng mối quan hệ 
- Thuyết phục và tư vấn khách hàng về các chính sách bán hàng và khuyến mãi của công ty. 
- Theo dõi và chăm sóc khách hàng hiện tại để đảm bảo sự hài lòng liên tục 
- Tham gia vào các hoạt động phát triển khách hàng và tiếp thị, như quảng cáo, trưng bày sản phẩm, hội chợ triển lãm... 
- Ghi chép thông tin liên lạc với khách hàng và phản hồi của họ 
- Báo cáo quản lý về tiến độ bán hàng, số liệu thống kê và các nhiệm vụ công việc khác theo yêu cầu. 
<|im_end|>


**Evalution**

In [ ]:
responses = [response1, response2, response3, response4, response5]

In [ ]:
answer1 = "Lương cơ bản cho vị trí này là 7000000vnd."
answer2 = "Địa điểm làm việc ở Hồ Chí Minh là số 7 đường Cộng Hòa, phường 4, Tân Bình."
answer3 = "Lương cơ bản 6 đến 12 triệu + thưởng doanh số + thưởng nóng"
answer4 = "Hồ Chí Minh: 239 Tân Sơn Nhì, P. Tân Sơn Nhì, Tân Phú"
answer5 = "Khảo sát thị trường, tìm kiếm khách hàng, giới thiệu sản phẩm đến khách hàng là các Shop mỹ phẩm, thẩm mỹ viện/spa/phòng khám da liễu;\nchăm sóc khách hàng cũ, phát triển khách hàng mới.\nTư vấn và giải đáp các thắc mắc của khách hàng\nĐề xuất các kế hoạch kinh doanh, phương án hỗ trợ khách hàng nhằm thúc đẩy doanh thu.\nDuy trì và phát triển mối quan hệ với khách hàng, xây dựng tệp khách hàng tiềm năng.\nTiếp nhận các chính sách bán hàng, marketing của công ty để tư vấn, thuyết phục khách hàng mua sản phẩm.\nCác công việc phát sinh theo yêu cầu của Quản lý."

In [ ]:
answers = []
answers.append(answer1)
answers.append(answer2)
answers.append(answer3)
answers.append(answer4)
answers.append(answer5)

In [ ]:
def evalution(responses, answers, embedding_path):

  embedding = HuggingFaceEmbeddings(model_name= embedding_path)
  similarity_list = []
  for i in range(len(responses)):
    embedding_response = embedding.embed_query(responses[i]['result'])
    embedding_answer = embedding.embed_query(answers[i])
    similarity = cosine_similarity([embedding_response], [embedding_answer])
    similarity_list.append(similarity)
  return similarity_list

similarity_list = evalution(responses, answers, embedding_path)
similarity_list

[array([[0.95736642]]),
 array([[0.98038478]]),
 array([[0.71670844]]),
 array([[0.62789655]]),
 array([[0.86534431]])]

In [ ]:
# average evalution
similarity_avg = sum(similarity_list) / len(similarity_list)
print(f"Similarity of question and response :{similarity_avg[0][0]:.2f}")

Similarity of question and response :0.83


**Prompt engineering**

In [ ]:
custom_prompt_template = """
Bạn là một trợ lý AI chuyên nghiệp trong lĩnh vực tuyển dụng. Nhiệm vụ của bạn là trả lời câu hỏi dựa trên thông tin được cung cấp trong ngữ cảnh một cách chính xác, ngắn gọn và dễ hiểu.
Nếu ngữ cảnh không đủ để trả lời, hãy trả lời: "Tôi không có đủ thông tin để trả lời."
----
Ngữ cảnh: {context}
Câu hỏi: {question}
----
Trả lời:
"""

In [ ]:
create_db_from_text(raw_text, embedding_path, vector_db_path)

From this text we have 4 chunks
Chunks: Hỗ trợ các phòng ban liên quan đến kỹ thuật.
Nắm rõ về chất liệu, phụ kiện (Cập nhật và đào tạo thêm).
Ưu tiên nhân sự có thể khảo sát 1 mình, có thể khảo sát ngoài giờ hành chính.
Bóc tách chi tiết sát với hình ảnh 3D.
Có đề xuất thay đổi chi tiết 2D so với 3D nếu thấy hợp lý hơn.

Number token of chunks:
Chunk 0 has 107 tokens
Chunk 1 has 112 tokens
Chunk 2 has 82 tokens
Chunk 3 has 117 tokens


In [ ]:
question = "Lương cơ bản là bao nhiêu ?"
response = final_result(question)
print(response['query'])
print(response['result'])

Lương cơ bản là bao nhiêu ?
Lương cơ bản của thợ hàn là 7000000vnd.


In [ ]:
question = "Địa điểm làm việc ở thành phố hồ chí minh là ở đâu ?"
response = final_result(question)
print(response['query'])
print(response['result'])

Địa điểm làm việc ở thành phố hồ chí minh là ở đâu ?
Địa điểm làm việc của thành phố Hồ Chí Minh nằm tại số 7 đường Cộng Hòa, phường 4, Tân Bình.
